---

# <center>*__Cryptography MasterClass By GROK__*

---

`[18 Nov 2025]` - Updated at : https://github.com/Harikalatheeswaran/Cryptography_MasterClass_By_GROK/blob/main/Cryptography_MasterClass_By_GROK.ipynb

```python
num = 255
print(f"{num:x}")  # ff (hex)
print(f"{num:b}")  # 11111111 (binary)
print(f"{num:o}")  # 377 (octal)
```

---
---

## *__Important things to remember -__*
1. > x << y : x * 2^y 
2. > x >> y : x / 2^y 
3. `If you want to extract last n bits of an integer, then perform bitwise AND of n & hex(2**n-1)`
   `n-1 because : say n is 3 digit binary then bin(hex(2*3)) = 1000` if we want 3 ones : we take 7 which is `111` 
4. 

---

## *__Functions in arsenal -__*
1. `int_to_bits`(n: int, length: int = 8) -> list
2. `bits_to_int`(bits: list) -> int
3. `xor_bits`(a: list, b: list) -> list
4. `int_to_bytes`(n: int) -> list
5. `bytes_to_int`(byte_list: list) -> int
6. `bytes_to_bits`(byte_list: list) -> list


---
---

## DAY 1 - "BITS ARE BULLETS"  
**Goal**: Tony will convert any number to binary bullets and back — with **MSB first** (left = big power, like we read), then glue two streams reversibly with XOR.

---

**Core Concept**  
Numbers are **stacks of coins**: 1, 2, 4, 8, 16… each twice the last.  
To make **5** → use 4 + 1 → so we need the **4-stack** and **1-stack**.  
In **8-bit binary**, we write from **biggest to smallest** (MSB first):  
`00000101` → position 7 to 0 → `5 = 4×1 + 1×1`  
Think of it like a **gun barrel**: left = heavy bullet (128), right = tiny pellet (1).  
**XOR is reversible glue**: same bullets cancel (`1 ⊕ 1 = 0`), different stick (`1 ⊕ 0 = 1`).  
It’s the **one-time pad** — unbreakable if secret is random and never reused.

---

**Physical Demo**  
Hold up 8 fingers **left to right** → finger 7 (128) to finger 0 (1).  
- Say: “5 needs finger 2 (4) and finger 0 (1).” → raise **left ring finger** and **right pinky**.  
- Now second hand: “Secret = 3 → fingers 1 and 0.”  
- **Where fingers match** → lower both. **Where one is up** → keep up.  
- Result: fingers 2 and 1 → `00000110` → `6`  
- Do again: `6 ⊕ 3 = 5` → **back!** Reversible glue.

---

**Code Module**  
```markdown
# bits.py - Hand-rolled binary artillery (MSB-first: human-readable)


In Python, the `>>` and `<<` operators are **bitwise shift operators**. Here's what they do:

***

### 🔹 `<<` — Left Shift Operator

*   **Syntax**: `a << b`
*   **Meaning**: Shifts the bits of `a` to the **left** by `b` positions.
*   **Effect**: Each left shift multiplies the number by 2.

**Example**:

```python
x = 5        # binary: 00000101
y = x << 1   # binary: 00001010 → decimal: 10
```

So `5 << 1` is `10`, and `5 << 2` is `20`.

***

### 🔹 `>>` — Right Shift Operator

*   **Syntax**: `a >> b`
*   **Meaning**: Shifts the bits of `a` to the **right** by `b` positions.
*   **Effect**: Each right shift divides the number by 2 (integer division).

**Example**:

```python
x = 20       # binary: 00010100
y = x >> 2   # binary: 00000101 → decimal: 5
```

So `20 >> 2` is `5`.

***

### 🧠 Analogy (for Tony-style learning)

*   **Left shift (`<<`)**: Like pushing marbles to the left on a tray — each move doubles the count.
*   **Right shift (`>>`)**: Like scooping marbles off the right — each move halves the count.

Would you like a small Jupyter Notebook-style demo with test vectors for these operators?

Operation example - 
```python
for i in range(1,6):
    print(i<<1) # Multiply i by 2
    print(1<<i) # 2 raised to the power i : 
    print("---------")
```


In [1]:

def int_to_bits(n: int, length: int = 8) -> list:
    """
    Convert integer n to list of bits (MSB first), padded to 'length' bits.
    Matches format: bin(5)[2:].zfill(8) → ['0','0','0','0','0','1','0','1']
    
    Physical: Left = 128 (heavy bullet), right = 1 (tiny pellet)
    Derivation: Start from highest power, check if n >= 2**i
    """
    if n < 0:
        raise ValueError("Only non-negative integers")
    bits = []
    for i in range(length - 1, -1, -1):    # i=7 down to 0
        bit = 1 if n >= (1 << i) else 0     # can we afford this coin? => the bit is made 1 only when the n>=2**i
        bits.append(bit)
        if bit:
            n -= (1 << i)                   # spend the coin => if the bit is 1, we reduce n by 2**i
    return bits

# _________________________________________________________________________________________________________________________________________________________

def bits_to_int(bits: list) -> int:
    """
    Convert MSB-first bit list back to integer.
    Physical: Sum value of raised fingers from left (big) to right (small)
    """
    total = 0
    for i, bit in enumerate(bits):
        if bit:
            total += (1 << (len(bits) - 1 - i))   # position from left
    return total

# _________________________________________________________________________________________________________________________________________________________

def xor_bits(a: list, b: list) -> list:
    """
    XOR two equal-length bit lists. Reversible glue.
    Physical: Same fingers up → cancel; different → keep one up
    """
    if len(a) != len(b):
        raise ValueError("Bit streams must match length")
    return [ai ^ bi for ai, bi in zip(a, b)]

# _________________________________________________________________________________________________________________________________________________________


# === TEST VECTORS (match real tools: bin(), OpenSSL hexdump style) ===
if __name__ == "__main__":
    # Test 1: 5 in 8 bits → [0,0,0,0,0,1,0,1]
    assert int_to_bits(5, 8) == [0,0,0,0,0,1,0,1], "5 failed"
    
    # Test 2: Round trip
    assert bits_to_int(int_to_bits(255, 8)) == 255
    
    # Test 3: XOR identity (A ⊕ B ⊕ B = A)
    a = int_to_bits(42, 8)
    b = int_to_bits(99, 8)
    assert xor_bits(xor_bits(a, b), b) == a
    
    # Test 4: Known XOR
    assert xor_bits([1,0,1,1], [1,1,0,1]) == [0,1,1,0]
    
    print("All bullets hit the target!")

All bullets hit the target!



**Tony’s Task**  
1. Open **Jupyter Notebook** → New → Python 3  
2. **Markdown cell**:  
   ```markdown
   # Day 1: BITS ARE BULLETS (MSB-First — Like We Read!)
   5 in 8 bits = [0,0,0,0,0,1,0,1] → left = 128, right = 1
   XOR = reversible glue: same cancels, different sticks
   ```  
3. **Code cell**: Type **every character** of `bits.py` above  
4. Run → fix typos → see `All bullets hit the target!`  
5. **New cell** — Tony’s Playground:  
   ```python
   print(int_to_bits(13, 8))        # → [0,0,0,0,1,1,0,1]
   print(bits_to_int([0,0,0,0,1,1,0,1]))  # → 13
   print(xor_bits([1,1,0,1], [0,1,1,0]))  # → [1,0,1,1]
   ```  
6. Save as `day01_bits_msb.ipynb`

---

**Night Drill**  
**Break & Fix**:  
- On paper: Write `1100` (12) and `1010` (10) → XOR → `0110` (6)  
- **Break code**: Change `1 << i` to `2 << i` → run → debug  
- Fix → remember: “`1 << i` = coin of size 2^i”

---

**Memory Bomb**  
**Rhyme to type in your sleep:**  
```
Left is heavy, right is light,  
Five is zero-zero-zero-zero-zero-one-zero-one,  
MSB first — like words we write,  
XOR glue undoes with second bite.
```

---

**Future Vision**  
With **MSB-first bits**, Tony can:  
- **Read hexdumps** like a pro: `0x0D = 00001101` → CR  
- **Spoof RFID**: Capture tag → flip bit 3 → clone badge  
- **Blink SOS**: `... --- ...` → `[0,0,0,0,0,1,1,0] ...`  
- **Build one-time pad**: XOR message with shared MSB-first key → **unbreakable**

---


In [ ]:
n = 9
length = 8
if n < 0:
    raise ValueError("Only non-negative integers")
bits = []
for i in range(length - 1, -1, -1):    # i=7 down to 0   
    if n >= (1 << i):
        bits.append(1)
        n = n - (1 << i)
        print(n)
    else:
        bits.append(0)

print(f"{bits=}")

| i | (1 << i) | n before | n >= (1<<i)? | bit | n after | printed  |
| - | -------- | -------- | ------------ | --- | ------- | -------- |
| 7 | 128      | 9        | No           | 0   | 9       | —        |
| 6 | 64       | 9        | No           | 0   | 9       | —        |
| 5 | 32       | 9        | No           | 0   | 9       | —        |
| 4 | 16       | 9        | No           | 0   | 9       | —        |
| 3 | 8        | 9        | Yes          | 1   | 1       | prints 1 |
| 2 | 4        | 1        | No           | 0   | 1       | —        |
| 1 | 2        | 1        | No           | 0   | 1       | —        |
| 0 | 1        | 1        | Yes          | 1   | 0       | prints 0 |


In [ ]:
for i in range(1,6):
    print(i<<1) # Multiply i by 2
    print(1<<i) # 2 raised to the power i
    print("---------")

---

# Day 2: BYTES ARE BOMBS
8 bits = 1 byte = 1 firecracker  
300 = [1, 44] → left = 256, right = 44  
Pack: n & 255 → shift >>8 → repeat  
Unpack: shift <<8 + next byte

## DAY 2 - "BYTES ARE BOMBS"  
**Goal**: Tony will pack 8 bits into a **byte bomb**, detonate it back to bits, and fuse multiple bytes into **messages** — the payload of every secret.

---

**Core Concept**  
A **byte** = **8 bits** = one **bomb casing**.  
We pack **MSB-first** (left = 128) so `00000101` = **5** fits in one byte.  
Think: **8 bullets in a clip**.  
- `int_to_bytes(5)` → one bomb: `[5]`  
- `int_to_bytes(300)` → **two bombs**: `[1, 44]` because 300 = 256×1 + 44  
We **prepend length** or **pad** so Tony can **rebuild the blast radius**.  
**Physical**: Each byte = one **firecracker**. String them → **fireworks**.

---

**Physical Demo**  
Hold up **two hands** = two bytes:  
- Left hand: `00000001` → 1 (tiny pop)  
- Right hand: `00101100` → 44 (bigger bang)  
- Together: `0000000100101100` → 300  
Now **light fuse from left**:  
> “First byte = 1 → shift left 8 → 256.  
> Add second byte = 44 → 256 + 44 = 300.”  
**Reverse**: Read bytes → rebuild number.  
**Boom → rebuild.**

---

**Code Module**  
```markdown
# bytes.py - Hand-rolled byte bombs (MSB-first, network order)
```




In [ ]:

def int_to_bytes(n: int) -> list:
    """
    Convert integer n to list of bytes (MSB-first, big-endian).
    Matches: n.to_bytes((n.bit_length()+7)//8, 'big')
    
    Physical: Pack number into 8-bit firecrackers, left = loudest.
    Derivation: While n > 0, take n % 256 = least significant byte, shift right 8.

    Clear explanation:
    >>  n & 0xFF extracts the least significant 8 bits (one byte) of n.
        n & 0xFF works because 0xFF in binary is 11111111, so the bitwise AND operation keeps only the last 8 bits of n.
        n & 0xFF is equivalent to n % 256.        

    >>  n >> 8 shifts n to the right by 8 bits, effectively removing the byte we just extracted.

    >>  n & 0FF extracts the last 8 bit of n and n >> 8 shifts n right by 8 bits.

    >>   We repeat this process until n becomes 0, collecting all bytes in reverse order.
    """
    if n < 0:
        raise ValueError("Only non-negative integers")
    if n == 0:
        return [0]
    bytes_list = []
    while n > 0:
        bytes_list.append(n & 0xFF)   # pluck 8 LSBs = one bomb
        n >>= 8                       # slide magazine left
    bytes_list.reverse()              # now MSB-first
    return bytes_list

# _________________________________________________________________________________________________________________________________________________________

def bytes_to_int_bin(out):
    '''
    Usin the bin funtion to convert bytes to integer
    '''
    r = [bin(i)[2:] for i in out]
    return int(''.join(r), 2)

# _________________________________________________________________________________________________________________________________________________________

def bytes_to_int(byte_list: list) -> int:
    """
    Convert list of bytes (MSB-first) back to integer.
    Physical: Detonate from left: each bomb adds value * 256^position
    """
    total = 0
    for byte_val in byte_list:
        total = (total << 8) + byte_val   # shift left 8, add new bomb
    return total

# _________________________________________________________________________________________________________________________________________________________

def bytes_to_bits(byte_list: list) -> list:
    """
    Explode bytes into flat MSB-first bit list.
    Physical: Open each firecracker → 8 bullets in order.
    """
    bits = []
    for byte_val in byte_list:
        bits.extend(int_to_bits(byte_val, 8))  # reuse Day 1 tool
    return bits

# _________________________________________________________________________________________________________________________________________________________

# === TEST VECTORS (match Python, OpenSSL, Wireshark) ===
if __name__ == "__main__":
    # Test 1: 5 → [5]
    assert int_to_bytes(5) == [5]
    
    # Test 2: 300 → [1, 44]
    assert int_to_bytes(300) == [1, 44]
    assert bytes_to_int([1, 44]) == 300
    
    # Test 3: 0 → [0]
    assert int_to_bytes(0) == [0]
    
    # Test 4: Round trip + bits
    n = 0xDEAD
    assert bytes_to_int(int_to_bytes(n)) == n
    assert bytes_to_bits([0xDE, 0xAD]) == [1,1,0,1,1,1,1,0, 1,0,1,0,1,1,0,1]
    
    print("All bombs detonated safely!")


All bombs detonated safely!


### __Amazing Discovery__

```
696969 converted to bytes is [10, 162, 137]
We can do : (10 << 16) ^ (162 << 8) ^ (137 << 0)
```
Code - 
```python
def bytes_to_int(byte_list : list) -> int:
    """
    Never mutate input lists as a good practice. 
    This is because other parts of the code might rely on the original order of the list.
    """
    # byte_list.reverse()
    # l = [byte_list[i] << (i*8) for i in range(len(byte_list))]
    output = 0
    for i, val in enumerate(byte_list[::-1]): # or reverserd(byte_list)
        output = output + (val << (i*8))
    return output

# 696969 converted to bytes is [10, 162, 137]
# (10 << 16) ^ (162 << 8) ^ (137 << 0) 
# instead of XOR we do normal addtion as sometimes XOR might become messy
x = [10, 162, 137]
bytes_to_int(x) # outputs 696969
```

In [ ]:
# Understanding the above code snippets:
print(int_to_bytes(1025))  # [4, 1]
# which is in a way - [00000100, 00000001] so that 1000000000 = 1025


[4, 1]


In [18]:
# 1 << 8 = 256 => adding 7 zeros after 1 gives 256.
print(f"{1 << 8}")  
print(f"{256 >> 8}")    



256
1


> **Note**: We **reuse** `int_to_bits(val, 8)` from Day 1 — Tony’s first tool now powers bigger bombs.

Wait — you reversed the list, then did:
byte[0] << 0 + byte[1] << 8 + byte[2] << 16 → perfect.
That’s exactly how the CPU does it under the hood.

```
[  10  ]  [ 162  ]  [ 137  ]   ← big-endian (network order)
  ^        ^         ^
  |        |         |
  |   shift by 16    |
  |        |         |
  +---- shift by 8 ---+
           |
       shift by 0
```

---

**Tony’s Task**  
1. **New Jupyter Notebook**: `day02_bytes.ipynb`  
2. **Markdown cell**:  
   ```markdown
   # Day 2: BYTES ARE BOMBS
   8 bits = 1 byte = 1 firecracker  
   300 = [1, 44] → left = 256, right = 44  
   Pack: n & 255 → shift >>8 → repeat  
   Unpack: shift <<8 + next byte
   ```  
3. **Code cell**: Type **every character** of `bytes.py`  
4. **Import Day 1**:  
   ```python
   # At top of notebook — Tony types this too!
   def int_to_bits(n, length=8): 
       bits = []
       for i in range(length-1, -1, -1):
           bit = 1 if n >= (1 << i) else 0
           bits.append(bit)
           if bit: n -= (1 << i)
       return bits
   ```  
5. Run all → `All bombs detonated safely!`  
6. **Playground cell**:  
   ```python
   print(int_to_bytes(1337))        # → [5, 57]
   print(bytes_to_int([5, 57]))     # → 1337
   print(bytes_to_bits([0xDE, 0xAD]))  # → full 16 bits
   ```

---

**Night Drill**  
**Break the bomb factory**:  
1. Change `n & 0xFF` → `n & 0xF` → run → debug  
   → “Only gets 4 bits!”  
2. Fix → say: “`0xFF = 11111111` = full byte mask”  
3. **Paper fuse**: Write `0xCAFE` → by hand:  
   - `0xCA = 202`, `0xFE = 254` → `[202, 254]`  
   - Now reverse: `202<<8 + 254` → 51966

---

**Memory Bomb**  
**Muscle Rhyme (clap & type):**  
```
Byte is eight, left is loud,  
Mask with FF, don’t be proud.  
Shift right eight, loop again,  
Reverse the list — big-endian!
```

---

**Future Vision**  
With **byte bombs**, Tony can:  
- **Send secret messages**: `"HI"` → `[72, 73]` → XOR with key → ciphertext  
- **Forge packets**: Craft TCP header → `[0x00, 0x50]` = port 80  
- **Crack weak keys**: Brute 2-byte key → 65,536 tries → door opens  
- **Build Bitcoin**: Transaction = list of bytes → sign → broadcast

---

Edited to check